## 1) 거래량 분석을 통한 대박 주 포착

시중에 출판된 차트 분석 책을 보면 단골손님으로 소개되는 주제가 바로 거래량 분석을 통한 대박 주 포착 알고리즘입니다. 그러한 책에서 주장하는 주요 요점은 ‘거래량은 조작하기 어려운 데이터이기 때문에 실제 세력들의 매집 패턴을 거래량을 통해 알 수 있다’는 것입니다. 알고리즘은 책마다 조금씩 차이가 있지만 대략 아래의 조건을 만족하는 종목을 찾으면 됩니다.

1. 대량 거래(거래량이 1,000% 이상 급증) 종목

2. 대량 거래 시점에서 PBR이 4보다 작아야 함

In [3]:
import win32com.client

In [4]:
# Creat object
instStockChart = win32com.client.Dispatch("CpSysDib.StockChart") 

# Set input values
instStockChart.SetInputValue(0, "A003540") # 종목코드: 대신증권
instStockChart.SetInputValue(1, ord('2')) # 요청구분: 개수로 요청
instStockChart.SetInputValue(4, 60) # 요청개수: 60개
instStockChart.SetInputValue(5, 8) # 필드: 거래량
instStockChart.SetInputValue(6, ord('D')) # 차트구분: 일별
instStockChart.SetInputValue(9, ord('1')) # 수정주가: Yes (default 0)

# Block request
instStockChart.BlockRequest()

# Get data
volumes = []
numData = instStockChart.GetHeaderValue(3)

for i in range(numData):
    volume = instStockChart.GetDataValue(0, i)
    volumes.append(volume)

print(volumes)

[88954, 94480, 89190, 213542, 260656, 99029, 112315, 43130, 135315, 97194, 93351, 65145, 208089, 265015, 140198, 78461, 115621, 85649, 76387, 71482, 66111, 20595, 51587, 35887, 34070, 52066, 66266, 39720, 50581, 45109, 77631, 48491, 125194, 100189, 81966, 51300, 36831, 49817, 49476, 63191, 95845, 155171, 92239, 140168, 81070, 144010, 114385, 92120, 182872, 249541, 91720, 108123, 171255, 80237, 123771, 136634, 94253, 113012, 120105, 81126]


이번에는 앞에서 구한 데이터를 이용해 거래량이 1,000% 급증했는지 확인하는 코드를 추가해보겠습니다. 앞의 코드에서 volumes라는 리스트에는 60일 치의 거래량이 저장됩니다. volumes[0]의 데이터가 최근 거래일의 거래량이므로 이를 제외한 나머지 59개의 거래량의 평균을 계산하겠습니다.

먼저 60일 치 거래량을 모두 더한 후 최근 거래량을 빼서 59일 치 거래량의 합을 계산합니다. 그리고 이 값을 59로 나눠서 평균 거래량을 계산합니다.

In [5]:
averageVolume = (sum(volumes) - volumes[0]) / (len(volumes) - 1)
averageVolume

100982.77966101695

In [6]:
volumes[0] # 가장 최근 거래량

88954

In [7]:
if volumes[0] > averageVolume*10:
    print("거래량 급등. 대박주")
else:
    print("일반주, 최근거래량/평균거래량: ", volumes[0]/averageVolume)

일반주, 최근거래량/평균거래량:  0.880882862391037


이를이용해 CheckVolume 함수를 만들고 해당하면 1, 해당하지 않으면 0을 return하게 만들어 전체 종목 중 거래량이 급등한 종목을 찾아낼 수도 있다. 

이 때, 함수의 인자를 어떻게 구성할지도 프로그램 효율성 측면에서 고려해야 한다. 

함수를 작성할 때는 함수의 인자를 어떻게 구성할지가 중요합니다. CheckVolumn 함수는 함수 내부에서 StockChart 인스턴스를 생성하지 않고 함수를 호출하는 곳에서 인스턴스를 생성하는 구조입니다. 만약 함수 내에서 인스턴스를 생성하도록 구현한다면 함수가 호출될 때마다 인스턴스가 생성됐다가 함수가 종료되면 인스턴스도 소멸해야 하므로 프로그램이 느려집니다.

(코드 생략)

참고로 위 코드를 실행했을 때 그림 10.22와 같은 경고 메시지가 나타날 수 있습니다. 이는 유가증권 시장의 전 종목에 대한 60일 치 거래량을 빠른 시간 동안 요청했기 때문입니다. 프로그래밍을 통해 데이터를 요청할 경우 사용자가 HTS로 데이터를 요청하는 것보다 짧은 시간 동안 많은 데이터를 요청할 수 있습니다. 사용자 입장에서는 API를 이용하는 것이 효율적이지만 증권사 서버는 이를 처리하는데 부담이 됩니다. 이러한 문제 때문에 보통 증권사에서는 회원의 계좌 등급에 따라 데이터를 처리하는데 제한을 두곤 합니다.

이 같은 문제가 발생하는 경우 가장 쉬운 해결책은 데이터를 요청할 때 각 요청 시점마다 강제로 약간의 지연 시간을 두는 것입니다. 파이썬의 time 모듈에는 sleep이라는 함수가 있는데 이 함수를 이용하면 일정 시간 동안 멈췄다가 다시 코드를 실행하게 할 수 있습니다.

다음은sleep(1)을 사용해 한 종목에 대한 데이터를 조회하고 1초 동안 멈춘 후에 다음 종목에 대한 데이터를 조회하도록 코드를 수정한 것입니다. 다만 이 경우 프로그램의 수행 시간이 길어진다는 단점이 있습니다.

In [ ]:
import time

# 중간 코드 생략 - 이 블록은 코드 실행 불가. 

for code in codeList:
    if CheckVolumn(instStockChart, code) == 1:
        buyList.append(code)
        print(code)
    time.sleep(1)

## 2) 업종별 PER 분석을 통한 유망 종목 찾기

더좋은알고리즘의개발은여러분에게맡기기로하고이번절에서는업종별PER 분석을통한유망종목을 찾는 알고리즘을 구현하는 과정을 설명하겠습니다. 먼저 업종별로 평균 PER을 계산하려면 업종별로 종목 코드 리스트를 만들어야 합니다.

CYBOS Plus의 도움말 파일을 참조하면 CpCodeMgr 클래스의 GetIndustryList 메서드를 사용하면 업종별 코드 리스트를 구할 수 있습니다. 먼저 CpCodeMgr 클래스의 인스턴스를 생성해 봅시다.

In [11]:
instCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
instCpCodeMgr

<COMObject CpUtil.CpCodeMgr>

업종별 코드 리스트를 구하기 위해 GetIndustryList 메서드를 호출합니다. GetIndustryList 메서드는 업종별 코드 리스트를 파이썬 튜플 형태로 반환합니다.

In [15]:
industryCodeList = instCpCodeMgr.GetIndustryList()
len(industryCodeList)

124

각 업종별 코드에 해당하는 업종명을 출력해보면 어떤 업종이 있는지 더 정확히 파악할 수 있습니다. 업종별 코드에 해당하는 업종명을 구하려면 GetIndustryName 메서드를 사용하면 됩니다. 이때 industryCodeList에는 많은 수의 업종별 코드가 존재하므로 반복문을 사용해 업종별 코드에 대한 업종명을 출력하면 됩니다.

In [16]:
for industryCode in industryCodeList:
    print(industryCode, instCpCodeMgr.GetIndustryName(industryCode))

001 종합주가지수
002 대형(시가총액)
003 중형(시가총액)
004 소형(시가총액)
005 음식료품
006 섬유,의복
007 종이,목재
008 화학
009 의약품
010 비금속광물
011 철강,금속
012 기계
013 전기,전자
014 의료정밀
015 운송장비
016 유통업
017 전기가스업
018 건설업
019 운수창고
020 통신업
021 금융업
022 은행
024 증권
025 보험
026 서비스업
027 제조업
130 KOSPI200 경기방어소비재
131 KOSPI200 중소형주지수
132 KOSPI200 산업재
133 KOSPI200 헬스케어
141 KOSPI200 건설
142 KOSPI200 중공업
143 KOSPI200 철강/소재
144 KOSPI200 에너지/화학
145 KOSPI200 정보기술
146 KOSPI200 금융
147 KOSPI200 생활소비재
148 KOSPI200 경기소비재
149 코스피 200 에너지화학 레버리지지수
150 미국달러선물 레버리지지수
151 미국달러선물지수
152 미국달러선물인버스지수
153 코스피 200 커버드콜 5% OTM
154 코스피 200 프로텍티브풋 OTM 5%
155 KOSPI200 동일가중지수
156 KOSPI100 동일가중지수
157 KOSPI50 동일가중지수
165 코스피 200 정보기술 레버리지지수
166 코스피 200 금융 레버리지지수
167 코스피 200 경기소비재 레버리지지수
170 KTOP 30
177 KOSPI200레버리지지수
178 F-KOSPI200인버스지수
179 KOSPI200선물지수
180 KOSPI200지수
181 KOSPI100지수
182 KOSPI50지수
184 코스피 200 TR
185 코스피 200 NTR
186 KRX-IHS Markit 코스피 200 예측 고배당 30
187 KRX-IHS Markit 코스피 200 예측 배당성장 30
190 코스피 200 고배당지수
191 코스피 200 저변동성지수
192 코스피 고배당 50지수
193 코스피 배당성장 50지수


유가증권시장에서 ‘음식료품’ 업종에 속하는 종목의 종목 코드 리스트를 구한 후 해당 리스트에 속하는 종목의 종목명을 출력해 봅시다. CYBOS Plus 도움말을 참조하면 GetGroupCodeList 메서드가 업종 코드에 해당하는 종목 코드 리스트를 반환하는 것을 알 수 있습니다.

예를 들어, 다음은 유가증권시장의 ‘음식료품’ 업종에 속하는 종목 코드 리스트를 구한 후 각 종목의 종목 코드와 종목명을 출력하는 코드입니다.

In [21]:
targetCodeList = instCpCodeMgr.GetGroupCodeList(5)

for code in targetCodeList:
    print(code, instCpCodeMgr.CodeToName(code))

print("\n Total of: ", len(targetCodeList))

A011150 CJ씨푸드
A011155 CJ씨푸드1우
A097950 CJ제일제당
A097955 CJ제일제당 우
A023150 MH에탄올
A005610 SPC삼립
A002140 고려산업
A003920 남양유업
A003925 남양유업우
A004370 농심
A001680 대상
A001685 대상우
A001790 대한제당
A001799 대한제당3우B
A001795 대한제당우
A001130 대한제분
A049770 동원F&B
A280360 롯데제과
A005300 롯데칠성
A005305 롯데칠성우
A002270 롯데푸드
A027740 마니커
A033920 무학
A000890 보해양조
A005180 빙그레
A003960 사조대림
A008040 사조동아원
A014710 사조씨푸드
A006090 사조오양
A145990 삼양사
A145995 삼양사우
A003230 삼양식품
A248170 샘표식품
A004410 서울식품
A004415 서울식품우
A136490 선진
A007310 오뚜기
A271560 오리온
A006980 우성사료
A002600 조흥
A264900 크라운제과
A26490K 크라운제과우
A036580 팜스코
A017810 풀무원
A000080 하이트진로
A000087 하이트진로2우B
A003680 한성기업
A101530 해태제과식품

 Total of:  48


이번에는 10.3.3절에서 다룬 내용을 바탕으로 ‘음식료품’ 업종의 평균 PER을 계산해보겠습니다. 지금까지 작성한 코드를 살펴보면 CpCodeMgr 클래스의 인스턴스를 사용해 유가증권시장에서 ‘음식료품’ 업종에 해당하는 종목의 종목 코드 리스트를 구했습니다.

각 종목의 PER 값은 CpSyDib 모듈의 MarketEye 클래스를 사용해 얻을 수 있었습니다. 벌써 기억이 잘 나지 않는 분들은 10.3.3절의 내용을 참고합니다. MarketEye 클래스를 사용하기 위해 먼저 인스턴스를 생성합니다.

In [19]:
instMarketEye = win32com.client.Dispatch('CpSysDib.MarketEye')

# Get PER
instMarketEye.SetInputValue(0, 67) # 필드: 67번 PER(float)
instMarketEye.SetInputValue(1, targetCodeList) # 종목코드 또는 종목코드배열. 최대 200종목까지 가능. 

instMarketEye.BlockRequest()

numStock = instMarketEye.GetHeaderValue(2) # PER 구해온 종목 수
numStock

48

In [22]:
# Get Avg. PER

sumPER = 0

for i in range(numStock):
    sumPER += instMarketEye.GetDataValue(0, i)

print('Avg PER: ', sumPER/numStock)

Avg PER:  33.452083150545754
